In [3]:
import pandas as pd
import numpy as np

# Necesidades de producto de las sucursales
deficit_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\deficit.csv')

# Inventario disponible de productos en otras sucursales distintas a la sucursal con deficit
inventario_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\inventario.csv')

# Matriz de traspasos, prioridad de la ruta entre sucursales, y elegibilidad de la sucursal
prioridades_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\matriz_traspasos.csv')

# Valor para decidir a quien se la da prioridad como sucursal (ordenada segun ventas totales, de mayor a menor; revisar orden)
orden_prioridad_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\sucursales_prioridad.csv')

# Ordenar los déficits por prioridad de sucursal
deficit_df = deficit_df.merge(orden_prioridad_df, on='sucursal')
deficit_df = deficit_df.sort_values(by='orden')

# /////////////////////////////////////////// Variables ////////////////////////////////////////////////////
kms_maximos_distancia = 320 # Establecer el valor máximo de kms de distancia entre sucursales
max_exced_permitido_AB = 0.2 # Establecer el porcentaje máximo permitido que una sucursal puede tomar del excedente de un producto clasificación A o B
max_inv_disp_C = 0.7 # Establecer el porcentaje máximo permitido que una sucursal puede tomar del inv disp de un producto clasificación C
# el inventario disponible que una sucursal puede tomar de otra, de un produco clasificación L es 100%.

# Filtrar matriz de traspasos, según kms máximos y elegibilidad
prioridades_df = prioridades_df[(prioridades_df['prioridad'] <= kms_maximos_distancia) & (prioridades_df['elegible'] == 1)]

# Ajustar el inventario disponible para reflejar la relación con el excedente
inventario_df['inventario_ajustado'] = inventario_df['inventario_disponible']
inventario_df['excedente_ajustado'] = inventario_df['excedente']

# Normalización de valores
prioridad_max = prioridades_df['prioridad'].max()  
clasificacion_dict = {'A': 1, 'B': 2, 'C': 3, 'L': 4}
inventario_max = inventario_df['inventario_ajustado'].max()
excedente_max = inventario_df['excedente_ajustado'].max()
clas_max = max(clasificacion_dict.values())

# Asignar los pesos específicos
w_P = 0.00 # Prioridad geográfica
w_C = 0.00 # Prioridad de clasificación ABC
w_I = 0.8 # Prioridad por inventario disponible
w_E = 0.2 # Prioridad por excedente
# /////////////////////////////////////////// Variables ////////////////////////////////////////////////////


def calcular_puntuacion(s_origen, s_destino, producto):
    # Obtener datos relevantes
    prioridad = prioridades_df[(prioridades_df['sucursal_origen'] == s_origen) & (prioridades_df['sucursal_destino'] == s_destino)]['prioridad'].values[0]
    clasificacion = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['clasificacion'].values[0]
    inventario_ajustado = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['inventario_ajustado'].values[0]
    excedente = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['excedente_ajustado'].values[0]
    costo_unitario = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['costo_unitario'].values[0]
    
    # Convertir clasificación a un valor numérico
    clasificacion_val = clasificacion_dict[clasificacion]
    
    # Normalización
    P_norm = (prioridad_max - prioridad + 1) / prioridad_max  # Prioridad geográfica menor es mejor (menos kms de distancia)
    C_norm = clasificacion_val / clas_max  # Máximo valor de clasificación ABC
    I_norm = inventario_ajustado / inventario_max
    E_norm = excedente / excedente_max
    
    # Calcular puntuación
    puntuacion = (w_P * P_norm) + (w_C * C_norm) + (w_I * I_norm) + (w_E * E_norm)
    return puntuacion, costo_unitario, clasificacion

def satisfacer_deficit(deficit_row):
       
    sucursal_destino = deficit_row['sucursal']
    producto = deficit_row['producto']
    unidades_necesarias = deficit_row['unidades_necesarias']

    # Filtrar sucursales elegibles y calcular puntuaciones
    sucursales_origen = prioridades_df[(prioridades_df['sucursal_destino'] == sucursal_destino) & (prioridades_df['elegible'] == 1)]['sucursal_origen'].unique()

    puntuaciones = []

    for s_origen in sucursales_origen:
        if not inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)].empty:
            puntuacion, costo_unitario, clasificacion = calcular_puntuacion(s_origen, sucursal_destino, producto)
            puntuaciones.append((s_origen, puntuacion, costo_unitario, clasificacion))

    # Ordenar sucursales por puntuación descendente
    puntuaciones.sort(key=lambda x: x[1], reverse=True)

    # Realizar traspasos hasta satisfacer el déficit
    traspasos = []
    unidades_totales_traspasadas = 0
    for s_origen, _, costo_unitario, clasificacion in puntuaciones:
        if unidades_necesarias <= 0:
            break
        print(f"Evaluando s_origen: {s_origen}, producto: {producto}, clasificacion: {clasificacion}")

        # Guardar el 80% del excedente inicial si la clasificación es 'A' o 'B'
        if clasificacion in ['A', 'B']:
            excedente_inicial = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'excedente_ajustado'].values[0]
            limite_excedente = int(excedente_inicial * max_exced_permitido_AB)
        else:
            limite_excedente = None

        # Calcular el límite de inventario ajustado solo una vez para clasificación 'C'
        if clasificacion == 'C':
            inventario_ajustado_inicial = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'].values[0]
            limite_inventario_ajustado = int(inventario_ajustado_inicial * max_inv_disp_C)
        else:
            limite_inventario_ajustado = None

        while unidades_necesarias > 0:
            excedente = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'excedente_ajustado'].values[0]
            inventario_ajustado = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'].values[0]
            print(f"Excedente: {excedente}, Inventario ajustado: {inventario_ajustado}, Unidades necesarias: {unidades_necesarias}")

            if excedente > 0:
                if clasificacion in ['A', 'B']:
                    unidades_traspaso = min(limite_excedente, unidades_necesarias)
                else:
                    unidades_traspaso = min(excedente, unidades_necesarias)
                traspasos.append((s_origen, sucursal_destino, producto, unidades_traspaso, costo_unitario))
                inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'excedente_ajustado'] -= unidades_traspaso
                inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'] -= unidades_traspaso
                unidades_necesarias -= unidades_traspaso
                unidades_totales_traspasadas += unidades_traspaso
                if clasificacion in ['A', 'B']:
                    limite_excedente -= unidades_traspaso
            elif inventario_ajustado > 0:
                if clasificacion == 'L':
                    unidades_traspaso = min(inventario_ajustado, unidades_necesarias)
                elif clasificacion == 'C':
                    unidades_traspaso = min(limite_inventario_ajustado, unidades_necesarias)
                else:
                    break
                print(f"Unidades traspaso (inventario ajustado): {unidades_traspaso}")
                if unidades_traspaso > 0:
                    traspasos.append((s_origen, sucursal_destino, producto, unidades_traspaso, costo_unitario))
                    inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'] -= unidades_traspaso
                    unidades_necesarias -= unidades_traspaso
                    unidades_totales_traspasadas += unidades_traspaso
                    if clasificacion == 'C':
                        limite_inventario_ajustado -= unidades_traspaso
                else:
                    break
            else:
                break
            if unidades_traspaso == 0:
                break

    deficit_residual = deficit_row['unidades_necesarias'] - unidades_totales_traspasadas
    return traspasos, deficit_residual

# Aplicar el proceso para cada déficit
todos_traspasos = []
deficits_residuales = []

for idx, row in deficit_df.iterrows():
    traspasos, deficit_residual = satisfacer_deficit(row)
    todos_traspasos.extend(traspasos)
    if deficit_residual > 0:
        deficits_residuales.append((row['sucursal'], row['producto'], deficit_residual))

# Crear el DataFrame con los traspasos realizados
traspasos_df = pd.DataFrame(todos_traspasos, columns=['sucursal_origen', 'sucursal_destino', 'producto', 'unidades_traspaso', 'costo_unitario'])
traspasos_df = traspasos_df[traspasos_df['unidades_traspaso'] != 0]

# Crear el DataFrame con los déficits residuales
deficits_residuales_df = pd.DataFrame(deficits_residuales, columns=['sucursal_destino', 'producto', 'unidades_no_cubiertas'])

# Agrupar por sucursal_origen, sucursal_destino y producto
deficits_residuales_df = deficits_residuales_df.groupby(['sucursal_destino', 'producto']).agg({
    'unidades_no_cubiertas': 'sum'
}).reset_index()

# Agrupar por sucursal_origen, sucursal_destino y producto
traspasos_df = traspasos_df.groupby(['sucursal_origen', 'sucursal_destino', 'producto']).agg({
    'unidades_traspaso': 'sum',
    'costo_unitario': 'mean'
}).reset_index()

deficits_residuales_df = deficits_residuales_df.sort_values(by=['sucursal_destino', 'producto'], ascending=[True, True])
traspasos_df = traspasos_df.sort_values(by=['producto', 'sucursal_destino', 'sucursal_origen'], ascending=[True, True, True])

# Exportar traspasos realizados y déficits residuales a archivos CSV
traspasos_df.to_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Desktop\traspasos_recomendados.csv', index=False)
deficits_residuales_df.to_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Desktop\traspasos_no_procesados.csv', index=False)
print("Listo.")


Evaluando s_origen: 005-A, producto: DODG0085202, clasificacion: C
Excedente: 0, Inventario ajustado: 11, Unidades necesarias: 14
Unidades traspaso (inventario ajustado): 7
Excedente: 0, Inventario ajustado: 4, Unidades necesarias: 7
Unidades traspaso (inventario ajustado): 0
Evaluando s_origen: 003-A, producto: DODG0085202, clasificacion: L
Excedente: 6, Inventario ajustado: 6, Unidades necesarias: 7
Excedente: 0, Inventario ajustado: 0, Unidades necesarias: 1
Evaluando s_origen: 001-A, producto: DODG0085202, clasificacion: L
Excedente: 2, Inventario ajustado: 2, Unidades necesarias: 1
Evaluando s_origen: 003-A, producto: DODG0086553, clasificacion: L
Excedente: 4, Inventario ajustado: 4, Unidades necesarias: 5
Excedente: 0, Inventario ajustado: 0, Unidades necesarias: 1
Evaluando s_origen: 058-A, producto: DODG0086553, clasificacion: L
Excedente: 2, Inventario ajustado: 2, Unidades necesarias: 1
Evaluando s_origen: 003-A, producto: DODG0086551, clasificacion: L
Excedente: 4, Inventar

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\diego.salinas\\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\\Desktop\\traspasos_recomendados.csv'